# Quantifying Crowded Trades(QCT): A Comprehensive Analysis
### qct_Pseudocode
### Collins Emezie Ugwuozor, Jayesh Chandra Gupta, Emmanuel Effiong Asuquo

In [1]:
#Import the neccessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
from pandas_datareader import data
import yfinance as yf
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
#% matplotlib inline

In [2]:
# Pick up table of S&P 500 listed companies from Wikipedia.
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

In [3]:
table = data[0]
table.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [4]:
table['GICS Sector'].unique()

array(['Industrials', 'Health Care', 'Information Technology',
       'Consumer Staples', 'Utilities', 'Financials',
       'Consumer Discretionary', 'Materials', 'Real Estate',
       'Communication Services', 'Energy'], dtype=object)

In [5]:
# Select sectors of S&P 500
industrials_sector = table[table['GICS Sector'] == 'Industrials'] # XLI
health_sector = table[table['GICS Sector'] == 'Health Care'] # XLV
tech_sector = table[table['GICS Sector'] == 'Information Technology'] # XLK
consumer_staples_sector = table[table['GICS Sector'] == 'Consumer Staples'] # XLP
utilities_sector = table[table['GICS Sector'] == 'Utilities'] # XLU
financial_sector = table[table['GICS Sector'] == 'Financials'] # XLF
consumer_discretionary_sector = table[table['GICS Sector'] == 'Consumer Discretionary'] # XLY
materials_sector = table[table['GICS Sector'] == 'Materials'] # XLB
real_estate_sector = table[table['GICS Sector'] == 'Real Estate'] # XLRE
communication_services_sector = table[table['GICS Sector'] == 'Communication Services'] # XLC *XTL
energy_sector = table[table['GICS Sector'] == 'Energy'] # XLE

In [6]:
financial_sector # XLF

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
9,AFL,Aflac,Financials,Life & Health Insurance,"Columbus, Georgia",1999-05-28,4977,1955
20,ALL,Allstate,Financials,Property & Casualty Insurance,"Northbrook, Illinois",1995-07-13,899051,1931
30,AXP,American Express,Financials,Consumer Finance,"New York City, New York",1976-06-30,4962,1850
31,AIG,American International Group,Financials,Property & Casualty Insurance,"New York City, New York",1980-03-31,5272,1919
34,AMP,Ameriprise Financial,Financials,Asset Management & Custody Banks,"Minneapolis, Minnesota",2005-10-03,820027,1894
...,...,...,...,...,...,...,...,...
457,USB,U.S. Bank,Financials,Diversified Banks,"Minneapolis, Minnesota",NaN,36104,1968
476,V,Visa Inc.,Financials,Transaction & Payment Processing Services,"San Francisco, California",2009-12-21,1403161,1958
485,WFC,Wells Fargo,Financials,Diversified Banks,"San Francisco, California",1976-06-30,72971,1852
493,WTW,Willis Towers Watson,Financials,Insurance Brokers,"London, United Kingdom",2016-01-05,1140536,2016


In [7]:
# Define dates for five year historical prices
start_date = "2018-01-01"
end_date = "2023-01-01"

In [8]:
# Sector ETFs for centrality across sectors
symbols = ['XLB', 'XLI', 'XLY', 'XLP', 'XLE', 'XLV', 'XLF', 'XLK', 'XTL', 'XLU', 'XLRE']
# Assets from financial sector (XLF) for centrality across assets
financial_assets = financial_sector['Symbol'].tolist()
print(financial_assets)

['AFL', 'ALL', 'AXP', 'AIG', 'AMP', 'AON', 'ACGL', 'AJG', 'AIZ', 'BAC', 'WRB', 'BRK.B', 'BLK', 'BX', 'BK', 'BRO', 'COF', 'CBOE', 'SCHW', 'CB', 'CINF', 'C', 'CFG', 'CME', 'CMA', 'DFS', 'EG', 'FDS', 'FITB', 'FIS', 'FI', 'FLT', 'BEN', 'GL', 'GPN', 'GS', 'HIG', 'HBAN', 'ICE', 'IVZ', 'JKHY', 'JPM', 'KEY', 'L', 'MTB', 'MKTX', 'MMC', 'MA', 'MET', 'MCO', 'MS', 'MSCI', 'NDAQ', 'NTRS', 'PYPL', 'PNC', 'PFG', 'PGR', 'PRU', 'RJF', 'RF', 'SPGI', 'STT', 'SYF', 'TROW', 'TRV', 'TFC', 'USB', 'V', 'WFC', 'WTW', 'ZION']


In [9]:
# Get the  "Adj Close" data from Yahoo Finance
df_sectors = yf.download(symbols, start=start_date, end=end_date)['Adj Close']
# Calculate daily returns and drop NaNs
df_sectors = df_sectors.pct_change()
df_sectors.dropna(inplace=True)

# Standardize the data and calculate covariance matrix
scaler = StandardScaler()
df_sectors_scaled = scaler.fit_transform(df_sectors)
df_sector_cov = np.cov(df_sectors_scaled.T)

df_sectors.head(10)

[*********************100%***********************]  11 of 11 completed


,XLB,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY,XTL
Date,,,,,,,,,,,
2018-01-03 00:00:00-05:00,0.007004,0.014976,0.005372,0.005386,0.008341,-0.000354,0.000305,-0.007856,0.009568,0.004591,0.000723
2018-01-04 00:00:00-05:00,0.008735,0.006036,0.009263,0.007318,0.005055,0.002831,-0.017094,-0.008304,0.001421,0.003279,0.005777
2018-01-05 00:00:00-05:00,0.008018,-0.000400,0.002824,0.006875,0.010517,0.004411,0.002174,-0.000389,0.008518,0.007922,0.002154
2018-01-08 00:00:00-05:00,0.001432,0.006002,-0.001408,0.004123,0.003771,0.002459,0.006817,0.009351,-0.003637,0.001179,0.009314
2018-01-09 00:00:00-05:00,-0.001589,-0.002519,0.007754,0.006415,-0.002555,-0.001402,-0.011388,-0.009844,0.011774,0.001963,-0.006957
2018-01-10 00:00:00-05:00,-0.006046,-0.001197,0.008395,-0.000127,-0.003465,-0.004913,-0.015567,-0.010916,-0.001746,-0.000685,-0.012580
2018-01-11 00:00:00-05:00,0.013126,0.020495,0.004856,0.012750,0.004082,-0.001410,-0.006325,-0.003744,0.004313,0.016270,0.013899
2018-01-12 00:00:00-05:00,0.001580,0.009650,0.008975,0.009317,0.005872,0.000353,-0.007320,-0.005737,0.007660,0.012923,0.009425
2018-01-16 00:00:00-05:00,-0.011674,-0.012658,-0.002737,-0.008981,-0.003892,0.004590,0.004809,-0.002189,0.004953,-0.007141,-0.011176


In [10]:
# Get the  "Adj Close" data from Yahoo Finance
df_financial_assets = yf.download(financial_assets, start=start_date, end=end_date)['Adj Close']

# Select the Top 10 assets based on ROI
roi = (df_financial_assets.iloc[-1] - df_financial_assets.iloc[0])/ df_financial_assets.iloc[0]
top_10_ROI = roi.sort_values(ascending = False).head(10)
roi_tickers = top_10_ROI.index.tolist()
# Get the  "Adj Close" data of top 10 from Yahoo Finance
df2_prices = yf.download(roi_tickers, start=start_date, end=end_date)['Adj Close']
# Calculate daily returns for Top ROI Stocks
df2_financial_assets = df2_prices.pct_change().dropna()

[*********************100%***********************]  72 of 72 completed

1 Failed download:
- BRK.B: No timezone found, symbol may be delisted
[*********************100%***********************]  10 of 10 completed


In [11]:
# Standardize the data and calculate covariance matrix of financial assets
scaler = StandardScaler()
df2_financial_assets_scaled = scaler.fit_transform(df2_financial_assets)
df2_financial_assets_cov = np.cov(df2_financial_assets_scaled.T)

df2_financial_assets.head(10)

,AJG,AON,BRO,BX,MA,MMC,MSCI,NDAQ,PGR,WRB
Date,,,,,,,,,,
2018-01-03 00:00:00-05:00,0.007560,0.015971,0.011355,0.014760,0.012573,0.013060,0.015194,0.011988,-0.000542,-0.008281
2018-01-04 00:00:00-05:00,0.009259,0.013699,0.002710,0.007879,0.012937,0.010436,0.015738,0.013263,-0.000181,0.004031
2018-01-05 00:00:00-05:00,0.004587,0.008492,0.009460,-0.007516,0.020730,0.006926,0.010405,0.006608,0.003796,-0.004158
2018-01-08 00:00:00-05:00,0.000630,0.000073,-0.003060,0.008179,0.003023,-0.010619,0.001353,0.009469,0.003061,-0.005039
2018-01-09 00:00:00-05:00,0.001259,0.002489,0.008057,0.013822,0.001444,0.000000,0.006681,0.002626,0.008438,-0.004486
2018-01-10 00:00:00-05:00,-0.000943,-0.001899,-0.006280,0.013041,0.002257,-0.010123,-0.002610,0.013222,-0.008723,-0.012211
2018-01-11 00:00:00-05:00,-0.002674,-0.010171,-0.000383,0.010240,0.006631,-0.001725,0.007551,-0.009479,-0.000359,0.009713
2018-01-12 00:00:00-05:00,0.007571,0.009462,0.004214,0.024906,0.008762,0.008887,0.008385,0.012056,0.005031,0.012097
2018-01-16 00:00:00-05:00,-0.007201,-0.014720,-0.000954,-0.010455,-0.000247,-0.005873,-0.011480,-0.012403,0.005899,-0.000288


#### Step 1: Calculate Asset Centrality

## Absorbtion Ratio $$AR = \frac{\sum_{i=1}^n \sigma^2E_i}{\sum_{j=1}^N \sigma^2A_j}$$

- $N$ = number of sectors
- $n$ = number of eigenvectors
- $\sigma^2E_i$ = variance of the $i^{th}$ eigenvector
- $\sigma^2A_j$ = variance of the $j^{th}$ sector

The fraction of the total variance that is "absorbed"/explained by a fixed number of eigenvectors

We will Calculate the Absorbsion Ratio(AR) for Assets and Sectors using:
- PCA
- Eigen decomposition of Covariance matrix
- Eigen values and sum of variance of original data

The absorption ratio:
- Measures the degree of concentration within a set of assets.
- Represents the portion of variability in asset returns that can be attributed to a specific number of factors in a Principal Component Analysis (PCA).
- When elevated, indicates a propensity for shocks to rapidly disseminate across a wide spectrum.

## Centrality  $$C_i = \frac {\sum_{j=1}^n \biggl( AR^j  \frac{| EV_i^j |}{\sum_{k=1}^N | EV_k^j |} \biggr)}{\sum_{j=1}^n AR^j} $$

- $C_i$ = sector's centrality score
- $AR^j$ = absorbtion ratio of the $j^th$ eigenvector
- $EV_i^j$ = absolute value of the exposure of the $i^th$ sector within the $j^th$ eigenvector
- $n$ = number of eigenvectors in the numerator of the absorbtion ratio
- $N$ = total number of sectors

In [12]:
def centrality_score(X, n=2):
    N = X.shape[1]
    pca_model = PCA(n_components=n)
    pca_model.fit(X)
    EV = pca_model.components_
    AR = pca_model.explained_variance_ratio_
    C_list = []
    for i in range(N):
    #loop
        
        return C_list 

To assess sector and asset centrality, we will follow these steps:
- Gather five years of historical daily returns for both sectors and individual assets.
- Assign weights to each historical return based on the square root of their respective market capitalization weights.
- Calculate the covariance matrix encompassing both sectors and assets.
- Determine sector centrality using the top two eigenvectors.


#### Step 2: Calculate Relative Value

## Relative Value 


$$RV =  (Price-to-Book Ratio)/(5-Year Average Price-to-Book Ratio)/(Average of Normalized Valuations Across All Sectors) $$

- $RV$ = Relative Value


To determine the relative value of an asset and sectors, we will follow these steps:
- ACROSS ASSETS:
- Calculate the price-to-book value ratio for each asset.
- Normalize the valuation of each asset by dividing it by its own 5-year average. This adjustment accounts for variations in valuations across assets.
- Further, divide each asset's normalized valuation by the average of the normalized valuations of all other assets. This yields a cross-sectional measure of relative value, facilitating a comparative assessment of relative worth
- ACROSS SECTORS:
- We will use the current price of the sector
- The price will be normalize by dividing it by the sector's 200 Day SMA
- We will further divide each sector's current market cap by market cap weighted average of the normalized value of all sectors to establish a cross-sectional relative value measure



#### Step 3: Combination of  Centrality and Relative Value

## Centrality Used in Combination with Relative Value

A combination of Centrality and Relative value:
- Centrality provides a means to identify market bubbles, yet it struggles to distinguish between the ascent and decline phases of a bubble.
- On the other hand, Relative value aids in discerning the rise and fall of a bubble but does not excel at pinpointing the existence of bubbles in the first place.
- By integrating these two tools, we can effectively pinpoint and differentiate bubbles while also distinguishing their upward trends from their downward trajectories.

